## Synthetic Data Generation

This notebook explores the generation of synthetic data to train the future position prediction model. 

### Function to generate a Background Image


In [2]:
import cv2
import numpy as np


def generate_background_image(width, height, num_shapes, shape_size):
    """
    Generate a simple background image with random shapes.

    Parameters:
    width (int): Width of the background image
    height (int): Height of the background image
    num_shapes (int): Number of shapes to draw
    shape_size (int): Size of each shape

    Returns:
    img (numpy array): Background image
    """
    img = np.zeros((height, width, 3), dtype=np.uint8)  # Initialize black background

    for _ in range(num_shapes):
        x = np.random.randint(0, width - shape_size)
        y = np.random.randint(0, height - shape_size)
        cv2.rectangle(
            img,
            (x, y),
            (x + shape_size, y + shape_size),
            (
                np.random.randint(0, 255),
                np.random.randint(0, 255),
                np.random.randint(0, 255),
            ),
            -1,
        )

    return img

Example Usage

In [ ]:
# Example usage
# width, height = 640, 480  # Dimensions of the background image
# num_shapes = 10  # Number of shapes to draw
# shape_size = 20  # Size of each shape
# background_image = generate_background_image(
#     width, height, num_shapes, shape_size
# )  # Generate background image
# cv2.imwrite(
#     "/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/data/synthetic/background/background1.png",
#     background_image,
# )  # Save background image

### Camera intrinsic matrix

1. Setting Up the Camera Projection
   First, define the parameters for the camera:

**Intrinsic matrix (K)**: Describes the camera's internal parameters (focal length, optical center).  
**Extrinsic matrix (R, T)**: Describes the camera's position and orientation in the world (rotation and translation).


In [ ]:
K = np.array([[1000, 0, 128], [0, 1000, 128], [0, 0, 1]])

### Function to project 3D points to 2D


In [ ]:
# Function to project 3D points to 2D
def project_points(points, K, R, T):
    """
    Project 3D points to 2D using the pinhole camera model.

    Parameters:
        points: Nx3 array of 3D points
        K: Intrinsic matrix
        R: Rotation matrix
        T: Translation vector
    """
    points_homogeneous = np.hstack((points, np.ones((points.shape[0], 1))))
    projection_matrix = K @ np.hstack((R, T.reshape(-1, 1)))
    points_2d_homogeneous = points_homogeneous @ projection_matrix.T
    points_2d = points_2d_homogeneous[:, :2] / points_2d_homogeneous[:, 2, np.newaxis]
    return points_2d

### Function to Generate 3D Linear Motion Dataset


In [ ]:
import cv2
import numpy as np
import os
import json
import random


# Function to generate 3D linear motion dataset
def generate_3d_linear_motion_dataset(num_frames, bg_img, output_dir):
    """
    Generate a synthetic dataset with 3D linear motion.

    Parameters:
        num_frames (int): Number of frames to generate
        bg_img (str): Path to the background image
        output_dir (str): Output directory to save the frames and annotations
    """
    os.makedirs(output_dir, exist_ok=True)

    # Rotation (identity matrix, no rotation for simplicity)
    R = np.eye(3)
    T_initial = np.array(
        [0, 0, 1000]
    )  # Initial translation (starting far from the camera)

    # Initialise the list to store the annotations
    annotations = []

    for frame_num in range(num_frames):
        T = T_initial + np.array([frame_num * 5, 0, -frame_num * 5])

        # Define the 3D position of the object (can be a simple representation)
        obj_3d_position = np.array(
            [[0, 0, 0]]
        )  # Object centered at origin in its local 3D coordinate space

        # Project the 3D points to 2D
        obj_2d_position = project_points(obj_3d_position, K, R, T).astype(int)[0]

        # Calculate 2D bounding box
        w, h = 50, 50
        bbox = (obj_2d_position[0] - w // 2, obj_2d_position[1] - h // 2, w, h)

        # Draw the bounding box on the background
        img = cv2.imread(bg_img)
        cv2.rectangle(
            img,
            (bbox[0], bbox[1]),
            (bbox[0] + bbox[2], bbox[1] + bbox[3]),
            (255, 0, 0),
            2,
        )

        # Save the frame
        frame_filename = f"frame_{frame_num:04d}.png"
        frame_filepath = os.path.join(output_dir, frame_filename)
        cv2.imwrite(frame_filepath, img)

        # Append annotation
        annotations.append(
            {"frame": frame_filename, "bbox": [int(coord) for coord in bbox]}
        )

    # Save the annotations to a JSON file
    annotations_filepath = os.path.join(output_dir, "annotations.json")
    with open(annotations_filepath, "w") as f:
        json.dump(annotations, f, indent=4)

In [ ]:
# Example usage
# bg_img_path = "/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/data/synthetic/background/background1.png"
# generate_3d_linear_motion_dataset(
#     num_frames=100,
#     bg_img=bg_img_path,
#     output_dir="/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/data/synthetic/3d_linear_motion_dataset_test1",
# )

### Function to Generate 3D Non-linear Motion Dataset


In [ ]:
def process_frame(
    frame_num,
    T_initial,
    base_size,
    bg_img,
    output_dir,
    z_min,
    z_max,
    frame_size,
    scale_factor,
):
    """
    Process a single frame by projecting 3D points into 2D and saving the image with a bounding box.

    Paramètres :
        frame_num: Frame number
        T_initial: Initial translation vector
        base_size: Base size of the bounding box
        bg_img: Path to background image
        output_dir: Directory for saving frames
        z_min: Minimum value authorised for the camera's z coordinate
        z_max: Maximum authorised value for the camera's z coordinate
        frame_size : Size of the video frame
        scale_factor: Scaling factor to slow down movement
    """
    try:
        # Rotation (identity matrix, no rotation for simplicity)
        R = np.eye(3)

        # Non-linear translation 
        t = frame_num * scale_factor
        """ T = T_initial + np.array(
            [100 * np.cos(t), 100 * np.sin(t), -frame_num * 3 * scale_factor]
        ) """
        T = T_initial + np.array(
            [
                100 * np.cos(t) + 50 * np.sin(2 * t),
                100 * np.sin(t) + 50 * np.cos(2 * t),
                -frame_num * 3 * scale_factor,
            ]
        )

        # Ensures that the Z coordinate is within a valid range
        T[2] = np.clip(T[2], z_min, z_max)

        # Defines the object's 3D position
        obj_3d_position = np.array([[0, 0, 0]])

        # Projecting 3D points in 2D
        obj_2d_position = project_points(obj_3d_position, K, R, T).astype(int)[0]

        # Keeps the object in the frame
        obj_2d_position[0] = np.clip(
            obj_2d_position[0], base_size // 2, frame_size[0] - base_size // 2
        )
        obj_2d_position[1] = np.clip(
            obj_2d_position[1], base_size // 2, frame_size[1] - base_size // 2
        )

        # Calculates the size of the 2D bounding box as a function of camera distance
        z = T[2]  # Z coordinate of the camera translation vector
        size_scaling_factor = base_size / (z / 1000)
        w = h = max(
            1, int(size_scaling_factor)
        )  # Ensures that the width and height are at least 1

        # Calculates the 2D bounding box
        bbox = (obj_2d_position[0] - w // 2, obj_2d_position[1] - h // 2, w, h)

        # Draw the bounding box on the background
        img = cv2.imread(bg_img)
        if img is None:
            raise FileNotFoundError(f"Image de fond {bg_img} non trouvée")
        cv2.rectangle(
            img,
            (bbox[0], bbox[1]),
            (bbox[0] + bbox[2], bbox[1] + bbox[3]),
            (255, 0, 0),
            2,
        )

        # Saves the frame
        frame_filename = f"frame_{frame_num:04d}.png"
        frame_filepath = os.path.join(output_dir, frame_filename)
        cv2.imwrite(frame_filepath, img)

        return {"frame": frame_filename, "bbox": [int(coord) for coord in bbox]}
    except Exception as e:
        print(f"Erreur lors du traitement de la frame {frame_num} : {e}")
        return {"frame": None, "bbox": []}

In [ ]:
def generate_3d_nonlinear_motion_dataset(
    num_frames,
    bg_img,
    output_dir,
    z_min=100,
    z_max=1000,
    frame_size=(640, 480),
    scale_factor=0.01,
):
    """
    Generate a synthetic dataset with 3D non-linear motion.

    Paramètres:
        num_frames (int): Frames number
        bg_img (str): Path to background image
        output_dir (str): Path to output folder
        z_min (int, optional): Minimum value authorised for the camera's z coordinate. Defaults to 100.
        z_max (int, optional): Maximum authorised value for the camera's z coordinate. Defaults to 1000.
        frame_size (tuple, optional): Size of the video frame. Defaults to (640, 480).
        scale_factor (float, optional): Scaling factor to slow down movement. Defaults to 0.01.
    """
    os.makedirs(output_dir, exist_ok=True)

    #  Initial translation (initial distance from the camera)
    T_initial = np.array([0, 0, z_max])
    base_size = 50  # Basic size of the bounding box when z = z_max

    # Processing each frame
    annotations = []
    for frame_num in range(num_frames):
        annotation = process_frame(
            frame_num,
            T_initial,
            base_size,
            bg_img,
            output_dir,
            z_min,
            z_max,
            frame_size,
            scale_factor,
        )
        if annotation["frame"]:
            annotations.append(annotation)

    # Save the annotations to a JSON file
    annotations_filepath = os.path.join(output_dir, "annotations.json")
    with open(annotations_filepath, "w") as f:
        json.dump(annotations, f, indent=4)

In [ ]:
# if __name__ == "__main__":
#     bg_img_path = "/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/data/synthetic/background/background1.png"
#     generate_3d_nonlinear_motion_dataset(
#         num_frames=10000,
#         bg_img=bg_img_path,
#         output_dir="/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/data/synthetic/3d_nonlinear_motion_dataset_test6",
#     )